In [ ]:
import numpy as np
from collections import namedtuple
from bokeh.io import output_notebook, show
from bokeh.models import BoxSelectTool
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.electronic_structure.core import Spin
from mmmg_tools.objects import (
    Wavefunction,
    Charge,
    Structure,
)

output_notebook()

In [ ]:
vr = Vasprun('vasprun.xml', parse_potcar_file=False)

In [ ]:
class range_get():
    def __init__(self, emin, emax):
        self.range_tuple = namedtuple('Energy_range','Emin Emax')
        self.init = self.range_tuple(emin, emax)
        self.range = self.range_tuple(emin, emax)
    def update(self, emin, emax):
        self.range = self.range_tuple(emin, emax)
x = range_get(vr.tdos.energies[0] - vr.efermi, 0)

def modify_doc(doc, x=x):
    data = {spin.name: list(density * spin.value) for spin, density in vr.tdos.densities.items()}
    data['energy'] = list(vr.tdos.energies.copy() - vr.efermi)

    source = ColumnDataSource(data=data)
        
    plt = figure()
    plt.add_tools(BoxSelectTool(dimensions="width"))
    plt.circle('energy', 'up', source=source, alpha=0, nonselection_alpha=0)
    plt.line('energy', 'up', source=source, selection_color='red', nonselection_alpha=.6)
    if vr.is_spin:
        plt.line('energy', 'down', source=source, selection_color='red', nonselection_alpha=.6)
    
    def callback(attr, old, new, x=x):
        if len(new) != 0:
            i_min = min(new)
            i_max = max(new)
            x.update(source.data['energy'][i_min], source.data['energy'][i_max])
        else:
            x.update(x.init[0], x.init[1])
    
    source.selected.on_change('indices', callback)
    
    doc.add_root(plt)
    
show(modify_doc)

### Create CHGCAR from selected energy range

In [ ]:
structure = Structure.from_structure(vr.structures[-1])
wave = Wavefunction.from_file('WAVECAR', structure=structure)

In [ ]:
kweight = np.multiply(vr.actual_kpoints_weights, 1)
charge = Charge.from_wav(wave, kweight, brange=x.range)
print(charge.data['total'].sum() / (np.prod(charge.data['total'].shape)))

In [ ]:
charge.write_file('CHGCAR')